In [1]:
import pandas as pd
import numpy as np
import json
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re, string, unicodedata
import nltk
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [2]:
data = pd.read_csv("/content/drive/MyDrive/NLP RES DATASET/tweetsdataset.csv")

In [3]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2744 entries, 0 to 2743
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  2744 non-null   object 
 1   id          2742 non-null   float64
 2   author_id   2742 non-null   float64
 3   created_at  2742 non-null   object 
 4   text        2742 non-null   object 
dtypes: float64(2), object(3)
memory usage: 107.3+ KB


In [4]:
data.head()

,Unnamed: 0,id,author_id,created_at,text
0,0,5.723415e+17,1.101148e+08,2015-03-02T10:23:41.000Z,Drasko they didn't cook half a bird you idiot ...
1,1,5.723405e+17,3.865021e+07,2015-03-02T10:19:37.000Z,Hopefully someone cooks Drasko in the next ep ...
2,2,5.723347e+17,2.587278e+09,2015-03-02T09:56:43.000Z,of course you were born in serbia...you're as ...
3,3,5.723327e+17,2.601525e+09,2015-03-02T09:48:33.000Z,These girls are the equivalent of the irritati...
4,0,4.464610e+17,9.306205e+08,2014-03-20T01:39:29.000Z,RT @YesYoureRacist: At least you're only a tin...


In [17]:
# API keys and tokens
consumer_key = 'Oh4LbNEHR0TaQvvPxNYKne64Z'
consumer_secret = '5hI7vh8NqPW9wDaZEHYFzdelvmi3bsPsDqd6ZggevXQ4hIQlNn'
access_token = '1250011119894192130-snFrMa3tdwHIhpnN8oBkGQxafKreZl'
access_secret = 'X9nrbtmDSlOOEF7nP3wBy7sapqR1NWayJESFMjAfbqvnK'

In [6]:
import requests

# Replace 'your_bearer_token' with your actual bearer token
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAGcGewEAAAAAzcKmkAjePkVErzFdomb4JFOa%2FDQ%3DcSiPEto1GtsT1TjFOFIxpXW3pDDqbqRxB1MzZWY91uynVKtIY8'

# Set up headers with bearer token
headers = {
    'Authorization': f'Bearer {bearer_token}'
}

# Set up API endpoint
url = 'https://api.twitter.com/2/tweets/search/recent'

# Set up query parameters
def tweetQuerry(que):
  query_params = {
      'query': que,
      'max_results': 50,
      'tweet.fields': 'text,created_at,public_metrics'
  }

# Send API request
  response = requests.get(url, headers=headers, params=query_params)
  return response

In [7]:
jsonDef = tweetQuerry('F1')
print(jsonDef.json())

{'data': [{'text': 'RT @F1: STARTING GRID\n\nPerez and Alonso sharing the front row, Leclerc and Verstappen battling through the field... a mouthwatering race in…', 'edit_history_tweet_ids': ['1637179361383129090'], 'public_metrics': {'retweet_count': 570, 'reply_count': 0, 'like_count': 0, 'quote_count': 0, 'impression_count': 0}, 'created_at': '2023-03-18T19:49:11.000Z', 'id': '1637179361383129090'}, {'text': 'pure l’account ufficiale vuole la battaglia E BATTAGLIA SIA DOMANI https://t.co/LkEU5znwNo', 'edit_history_tweet_ids': ['1637179360309387265'], 'public_metrics': {'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0, 'impression_count': 0}, 'created_at': '2023-03-18T19:49:11.000Z', 'id': '1637179360309387265'}, {'text': 'RT @elmejordelresto: Verstappen cuando mañana se mande un Spa 2.0\n\n#F1 #SaudiArabianGP #verstappen https://t.co/sQ9WS7An7M', 'edit_history_tweet_ids': ['1637179358027628547'], 'public_metrics': {'retweet_count': 44, 'reply_count': 0, 'like_

In [10]:
print(type(jsonDef))
print(jsonDef.text)
csv_string = jsonDef.text

# Read the string into a DataFrame
finDf = pd.read_csv(pd.compat.StringIO(csv_string))

finDf


<class 'requests.models.Response'>
{"data":[{"text":"RT @F1: STARTING GRID\n\nPerez and Alonso sharing the front row, Leclerc and Verstappen battling through the field... a mouthwatering race in…","edit_history_tweet_ids":["1637179361383129090"],"public_metrics":{"retweet_count":570,"reply_count":0,"like_count":0,"quote_count":0,"impression_count":0},"created_at":"2023-03-18T19:49:11.000Z","id":"1637179361383129090"},{"text":"pure l’account ufficiale vuole la battaglia E BATTAGLIA SIA DOMANI https://t.co/LkEU5znwNo","edit_history_tweet_ids":["1637179360309387265"],"public_metrics":{"retweet_count":0,"reply_count":0,"like_count":0,"quote_count":0,"impression_count":0},"created_at":"2023-03-18T19:49:11.000Z","id":"1637179360309387265"},{"text":"RT @elmejordelresto: Verstappen cuando mañana se mande un Spa 2.0\n\n#F1 #SaudiArabianGP #verstappen https://t.co/sQ9WS7An7M","edit_history_tweet_ids":["1637179358027628547"],"public_metrics":{"retweet_count":44,"reply_count":0,"like_count":0,"quo

AttributeError: ignored